<h1>Process & ML TIME</h1>

Process info from Monmouth University 2020 national poll

In [135]:
import os 
import numpy as np
import pandas as pd 

from sklearn.model_selection import train_test_split
import os
import numpy as np
import pandas as pd

from utils import * 
from sklearn.svm import SVC
from sklearn.metrics import classification_report, f1_score, auc, roc_auc_score, roc_curve, precision_recall_curve, precision_score, recall_score, accuracy_score, confusion_matrix, RocCurveDisplay, PrecisionRecallDisplay, average_precision_score
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np 
import os 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, accuracy_score
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from sklearn.datasets import load_iris, make_classification
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.model_selection import cross_validate
from sklearn import svm
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
from xgboost import XGBRegressor, XGBClassifier

from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

from sklearn.compose import ColumnTransformer

In [136]:
propensity= pd.read_csv("../data/output11.csv")

<h2>Rename Columns</h2>

In [137]:
def recodeCols(df):

    cols = df.columns
    # vote choice
    if '12_2' not in cols:
        # for march 2020 data
        df['vote_choice_recoded'] = df.apply(lambda s: np.where(s['vote_choice'] in [6.0, 8.0, 9.0, 7.0], s['vote_choice_undecided'], s['vote_choice']), axis = 1)
        df['vote_choice_recoded'] = df['vote_choice_recoded'].apply(lambda s: np.where(s in [3.0, 9.0], 3.0, s))
    else:
        # for june 2020 data
        df['vote_choice'] = df.apply(lambda s: np.where(pd.isnull(s['12_1']), s['12_2'], s['12_1']), axis = 1)
      
        df['vote_choice_recoded'] = df.apply(lambda s: np.where(s['vote_choice'] in [6.0, 8.0, 9.0, 7.0], s['vote_choice_undecided'], s['vote_choice']), axis = 1)
        df['vote_choice_recoded'] = df['vote_choice_recoded'].apply(lambda s: np.where(s in [3.0, 4.0, 9.0], 3.0, s))
    # party recode
    df['party_recoded'] = df.apply(lambda s: np.where(s['party'] in [4.0, 9.0], s['party_unaffiliated'], s['party']), axis = 1)
    df['party_recoded'] = df['party_recoded'].apply(lambda s: np.where(s in [3.0, 9.0], 3.0, s))

    # age recode
    df['age_recoded'] = df.apply(lambda s: np.where(not pd.isnull(s['age_bin']), s['age_bin'], s['age']), axis = 1)

    df['age_recoded'] = df['age_recoded'].apply(lambda s: np.where(s >= 18 and s <= 34 and not pd.isnull(s), 1.0, np.where(s >= 35 and s <= 54, 2.0, 3.0)))   

    # approval ratings 
    df['approve_trump'] = df['approve_trump'] == 1
    df['approve_biden'] = df['approve_biden'] == 1 

    # race, overwrite if hispanic/latino
    df['latino'] = df['latino'] == 1

    def recode_race(s):
        if s['latino'] == 1 or s['race'] == 4: 
            return 4
        elif s['race'] != 5 and s['race'] != 9:
            return s['race']
        else:
            return 9
    df['race_recoded'] = df.apply(recode_race, axis = 1)

    # gender => male = True, female = False
    df['male'] = df['gender'] == 1

    # registered to vote

    df['registered_vote'] = df['registered_vote'] == 1


    # True if they think Trump is focused on important issues, False otherwise
    if 'focused_imp_issues' in cols:
        df['focused_imp_issues'] = df['focused_imp_issues'] == 1

    # combine some of the similar household concern categories 
    if 'top_household_concern' in cols:
        df['top_household_concern'] = df['top_household_concern'].apply(lambda s: np.where(s in [6.0, 7.0], 6.0, s))
        df['top_household_concern'] = df['top_household_concern'].apply(lambda s: np.where(s in [10.0, 11.0], 10.0, s))

    # turn into 1 = no college, 3 = college
    def recode_education(s):
        if s <= 3:
            return 1
        else:
            return 3
    df['education_recoded'] = df['education'].apply(recode_education)
    # combine into liberal, conservative, moderate, or other
    def recode_political_leaning(s):
        if s <= 2:
            return 1
        elif s == 4 or s == 9:
            return 2
        elif s == 3:
            return 3
        else:
            return 4
    df['political_leaning'] = df['political_leaning'].apply(recode_political_leaning)

    # Do you feel optimistic or pessimistic about the 2020 presidential election? 1 or 2 = optimistic of some sort, else pessimistic of some sort
    df['optimistic'] = df['optimistic'] <= 2

    # confidence in trump stamina to be president, under 2 is confident, over 2 is not confident
    if 'trump_stamina' in cols:
        df['trump_stamina'] = df['trump_stamina'] <= 2
    # confidence in biden stamina to be president, under 2 is confident, over 2 is not confident
    if 'biden_stamina' in cols:
        df['biden_stamina'] = df['biden_stamina'] <= 2
    ### https://www2.census.gov/geo/pdfs/maps-data/maps/reference/us_regdiv.pdf
    def recode_region(s):
        new_england = [9.0, 23.0, 25.0, 33.0, 44.0, 50.0, 34.0, 36.0, 42.0]
        midwest = [18.0, 17.0, 26.0, 39.0, 55.0, 19.0, 20.0, 27.0, 29.0, 31.0, 38.0, 46.0]
        south = [10.0, 11.0, 12.0, 13.0, 24.0, 37.0, 45.0, 51.0, 54.0, 1.0, 21.0, 28.0, 47.0, 5.0, 22.0, 40.0, 48.0]
        west = [4.0, 8.0, 16.0, 35.0, 30.0, 49.0, 32.0, 56.0, 2.0, 6.0, 15.0, 41.0, 53.0]
        
        if s in new_england:
            return 1
        elif s in midwest:
            return 2 
        elif s in south:
            return 3
        elif s in west:
            return 4
        
    df['region'] = df['state'].apply(recode_region)

    # merge propensities
    df['age_recoded'] = df['age_recoded'].astype(int)
    df_result = pd.merge(df, propensity, 
                        on=['male', 'race_recoded', 'education_recoded', 'age_recoded', 'STATEFIP'], 
                        how='left')

    # Renaming the merged column for clarity
    df['propensity'] = df_result['mrp_subgroup_estimate']
    df = df.dropna(subset = ['vote_choice_recoded'], how = 'all')

    return df

In [138]:
nat_2020 = pd.read_csv("../data/national_march_2020/MUP213_NATL_archive.tab", sep = "\t")
nat_2020 = nat_2020.rename(columns = {
        'Q3': 'top_household_concern',
        'QD1': 'registered_vote',
        'Q11': 'likely_to_vote', 
        'Q12': 'vote_choice', 
        'Q12B': 'vote_choice_undecided', 
        'Q13': 'approve_trump',
        'Q14': 'approve_biden',
        'Q15': 'optimistic',
        'Q16': 'elec_enthusiasm',
        'Q17': 'economic_situation',
        'Q19': 'focused_imp_issues',
        'QD2': 'party',
        'QD2A': 'party_unaffiliated', 
        'QD3': 'political_leaning', 
        'QD4': 'education',
        'QD5': 'age', 
        'QD5A': 'age_bin', 
        'QD7': 'latino', 
        'QD8': 'race', 
        'QD10': 'gender', 
        'QD11': 'state'
})

nat_2020 = nat_2020.drop(columns = [c for c in nat_2020.columns if 'Q' in c])
nat_2020['STATEFIP'] = nat_2020['state']
nat_2020 = recodeCols(nat_2020)
third_party_mar = nat_2020[nat_2020['vote_choice_recoded'] == 3.0]

nat_2020_pre_dummies = nat_2020.copy()


In [139]:
nat_2020_june = pd.read_csv("../data/national_june_2020/MUP218_NATL_archive_full.tab", sep = "\t")
nat_2020_june = nat_2020_june.rename(columns = {
        # 'Q3': 'top_household_concern',
        'QD1': 'registered_vote',
        'Q11': 'likely_to_vote', 
        'Q12': 'vote_choice', 
        'Q12_1': '12_1',
        'Q12_2': '12_2',
        'Q12B': 'vote_choice_undecided', 
        'Q15': 'approve_trump',
        'Q16': 'approve_biden',
        'Q19': 'optimistic',
        'Q21': 'elec_enthusiasm',
        'Q3': 'economic_situation',
        # 'Q19': 'focused_imp_issues',
        'Q17': 'trump_stamina',
        'Q18': 'biden_stamina',
        'QD2': 'party',
        'QD2A': 'party_unaffiliated', 
        'QD3': 'political_leaning', 
        'QD4': 'education',
        'QD5': 'age', 
        'QD5A': 'age_bin', 
        'QD7': 'latino', 
        'QD8': 'race', 
        'QD10': 'gender', 
        'QD11': 'state'
})
nat_2020_june = nat_2020_june.drop(columns = [c for c in nat_2020_june.columns if 'Q' in c])
nat_2020_june['STATEFIP'] = nat_2020_june['state']
nat_2020_june = recodeCols(nat_2020_june)
nat_2020_june.drop(columns=['12_1', '12_2', 'vote_choice', 'vote_choice_undecided'], inplace = True)
third_party_june = nat_2020_june[nat_2020_june['vote_choice_recoded'] == 3.0]


In [140]:
nat_2020_aug = pd.read_csv("../data/national_aug_2020/MUP222_NATL_archive_full.tab", sep = "\t")
nat_2020_aug = nat_2020_aug.rename(columns = {
        'Q3': 'top_household_concern',
        'QD1': 'registered_vote',
        'Q13': 'likely_to_vote', 
        'Q14': 'vote_choice', 
        # 'Q14_1': '14_1',
        # 'Q14_2': '14_2',
        'Q14B': 'vote_choice_undecided', 
        'Q17': 'approve_trump',
        'Q18': 'approve_biden',
        'Q21': 'optimistic',
        'Q23': 'elec_enthusiasm',
        # 'Q3': 'economic_situation',
        # 'Q19': 'focused_imp_issues',
        # 'Q17': 'trump_stamina',
        # 'Q18': 'biden_stamina',
        'QD2': 'party',
        'QD2A': 'party_unaffiliated', 
        'QD3': 'political_leaning', 
        'QD4': 'education',
        'QD5': 'age', 
        'QD5A': 'age_bin', 
        'QD7': 'latino', 
        'QD8': 'race', 
        'QD10': 'gender', 
        'QD11': 'state'
})
nat_2020_aug = nat_2020_aug.drop(columns = [c for c in nat_2020_aug.columns if 'Q' in c])
nat_2020_aug['STATEFIP'] = nat_2020_aug['state']

nat_2020_aug = recodeCols(nat_2020_aug)

nat_2020_aug.columns
nat_2020_aug.drop(columns=['vote_choice', 'vote_choice_undecided'], inplace = True)

<h2>Add Dummy Variables</h2>

In [141]:
categoricals = ['top_household_concern', 'registered_vote', 'region','economic_situation','likely_to_vote', 'education_recoded','elec_enthusiasm', 'political_leaning', 'race_recoded', 'party_recoded', 'age_recoded']
june_categoricals = ['registered_vote', 'region','economic_situation','likely_to_vote',  'education_recoded','elec_enthusiasm', 'political_leaning','race_recoded', 'party_recoded', 'age_recoded']
aug_categoricals = ['registered_vote', 'region','top_household_concern','likely_to_vote',  'education_recoded','elec_enthusiasm', 'political_leaning','race_recoded', 'party_recoded', 'age_recoded']

In [142]:
def dummyGrabber(df, cats):
    for category in cats:
        df[category] = df[category].astype('category')
    dummies = pd.get_dummies(df[cats])
    df = df[[c for c in df.columns if c not in cats]]
        
    df = pd.concat([df, dummies], axis = 1)
    
    return df

In [143]:
def checkVoteRecode(df):
    df = df[df['vote_choice_recoded'] != 3.0]
    # change trump to be 1 and biden to be 0
    # biden originally encoded as 2 and trump as 1. 
    # drop if vote choice recorded is null
    df = df.dropna(subset = ['vote_choice_recoded'])
    # vote choice recoded to int
    df['vote_choice_recoded'] = df['vote_choice_recoded'].astype(int)
    df['vote_choice_recoded'] = df['vote_choice_recoded'].apply(lambda s: np.where(s == 1, 1, 0))


    return df

In [144]:
nat_2020_june = checkVoteRecode(nat_2020_june)
nat_2020_aug = checkVoteRecode(nat_2020_aug)
nat_2020 = checkVoteRecode(nat_2020)

In [145]:
march_dummies = dummyGrabber(nat_2020, categoricals)
nat_2020_clean = march_dummies.drop(columns = ['RESPID', 'PHTYPE', 'age', 'age_bin', 'party', 'party_unaffiliated', 'gender', 'latino', 'race', 'education',])
nat_2020_clean
nat_2020_clean2= nat_2020_clean.drop(columns=['FINALWGT', 'vote_choice', 'vote_choice_undecided','state'])
nat_2020_clean2.columns

Index(['approve_trump', 'approve_biden', 'optimistic', 'focused_imp_issues',
       'STATEFIP', 'vote_choice_recoded', 'male', 'propensity',
       'top_household_concern_1.0', 'top_household_concern_2.0',
       'top_household_concern_3.0', 'top_household_concern_4.0',
       'top_household_concern_5.0', 'top_household_concern_6.0',
       'top_household_concern_8.0', 'top_household_concern_9.0',
       'top_household_concern_10.0', 'top_household_concern_12.0',
       'top_household_concern_13.0', 'top_household_concern_14.0',
       'top_household_concern_15.0', 'top_household_concern_16.0',
       'top_household_concern_17.0', 'top_household_concern_19.0',
       'top_household_concern_20.0', 'top_household_concern_28.0',
       'top_household_concern_29.0', 'top_household_concern_30.0',
       'registered_vote_True', 'region_1.0', 'region_2.0', 'region_3.0',
       'region_4.0', 'economic_situation_1', 'economic_situation_2',
       'economic_situation_3', 'economic_situation_9', 

In [146]:
june_dummies = dummyGrabber(nat_2020_june, june_categoricals)
nat_2020_june_clean = june_dummies.drop(columns = ['RESPID', 'PHTYPE', 'age', 'age_bin', 'party', 'party_unaffiliated', 'gender', 'latino', 'race', 'education',])
nat_2020_june_clean
nat_2020_june_clean2 = nat_2020_june_clean.drop(columns=['FINALWGT', 'state'])

In [147]:
aug_dummies = dummyGrabber(nat_2020_aug, aug_categoricals)
nat_2020_aug_clean = aug_dummies.drop(columns = ['RESPID', 'PHTYPE', 'age', 'age_bin', 'party', 'party_unaffiliated', 'gender', 'latino', 'race', 'education',])
nat_2020_aug_clean
nat_2020_aug_clean2 = nat_2020_aug_clean.drop(columns=['FINALWGT', 'state'])

# MACHINE LEARNING SETUP

In [148]:
models = [
  

    # ("Logistic Regression Classification", LogisticRegression(), {'C': [1, 2]}),

    ("Random Forest Classification", RandomForestClassifier(), {'n_estimators': [100,150,200]}),
    ("Random Forest Classification2", RandomForestClassifier(min_samples_split = 3,  n_estimators=75), {'n_estimators': [100,150,200]}), 
    ("Gradient Boosting Classification", GradientBoostingClassifier(criterion='squared_error', n_estimators=440, max_depth=5, learning_rate=0.012, random_state=0, max_features='sqrt'), {'n_estimators': [100,150,200]}),
    ("Gradient Boosting Classification2", GradientBoostingClassifier(criterion='squared_error', n_estimators=100, max_depth=7, learning_rate=0.2, random_state=0, max_features='sqrt'), {'n_estimators': [100,150,200]}),
    ("Gradient Boosting Classification3", GradientBoostingClassifier( n_estimators=375, max_depth=3, learning_rate=0.02, random_state=0, max_features=2), {'n_estimators': [100,150,200]}),
]

In [149]:
def modelFunc(X_train, y_train, X_test, y_test, data_dummies, models):
    best_f1 = 0
    for model in models:
    
        name = model[0]
        print(model[0])
        clf = model[1]
      
        grids =  GridSearchCV(clf, param_grid={}, scoring='f1_weighted' ,cv=5)
        grids.fit(X_train, y_train)
        y_preds = grids.best_estimator_.predict(X_test)
        if f1_score(y_test, y_preds, average='weighted') >= best_f1:
            new_model = grids.best_estimator_
        #print('train result', result['test_score'])
        grids.best_score_
        # print('mean_score:', -np.mean(result['test_score']))
        # y_preds = grids.best_estimator_.predict(X_test)

        print(f1_score(y_test, y_preds, average='weighted'))
    new_model.fit(X_train, y_train)
    y_preds = new_model.predict(X_test)
    X_test_copy = X_test.copy()
    X_test_copy['predicted_vote'] = y_preds
    X_test_copy['actual_vote'] = y_test
    X_test_copy_filtered = X_test_copy[['approve_trump', 'approve_biden', 'optimistic','male',  'race_recoded_1', 'race_recoded_2', 'race_recoded_3', 'race_recoded_4',
       'race_recoded_9', 'party_recoded_1.0',
       'party_recoded_2.0', 'age_recoded_1', 'age_recoded_2', 'age_recoded_3', 'predicted_vote','actual_vote', 'education_recoded_3', 'education_recoded_1']]

    filtered_nat_df = data_dummies.loc[X_test_copy_filtered.index].drop(columns= ['RESPID', 'PHTYPE', 'age', 'age_bin', 'party', 'party_unaffiliated', 'gender', 'latino', 'race', 'education', 'vote_choice_recoded', 'state'])
    filtered_nat_df['predicted_vote'] = X_test_copy['predicted_vote']
    filtered_nat_df['actual_vote'] = X_test_copy['actual_vote']
    
    df_final_pred = filtered_nat_df.groupby(['age_recoded','race_recoded', 'male','education_recoded'], )['predicted_vote'].value_counts().unstack(fill_value=0)
    #filtered_nat_2020.groupby(['age_recoded','race_recoded', 'male','education_recoded'])['predicted_vote'].value_counts().unstack(fill_value=0)
    return df_final_pred

In [150]:
# nat_2020_clean2.notnull().sum()
nat_2020_clean2 = nat_2020_clean2.dropna()
X = nat_2020_clean2.drop(columns = ['vote_choice_recoded', 'STATEFIP'])
y = nat_2020_clean2['vote_choice_recoded']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.6, random_state = 11)
mar_pred = modelFunc(X_train, y_train, X_test, y_test, nat_2020_pre_dummies, models)
# nat_2020_june_clean2.dropna(subset = ['STATEFIP'], how = 'all', inplace = True)
X = nat_2020_june_clean2.drop(columns = ['vote_choice_recoded', 'STATEFIP'])
y = nat_2020_june_clean2['vote_choice_recoded']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.6, random_state = 11)
june_pred = modelFunc(X_train, y_train, X_test, y_test, nat_2020_june, models)

nat_2020_aug_clean2 = nat_2020_aug_clean2.dropna()
X = nat_2020_aug_clean2.drop(columns = ['vote_choice_recoded', 'STATEFIP'])
y = nat_2020_aug_clean2['vote_choice_recoded']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.6, random_state = 11)
aug_pred = modelFunc(X_train, y_train, X_test, y_test, nat_2020_aug, models)

Random Forest Classification
0.9292374072071868
Random Forest Classification2
0.9386833392468367
Gradient Boosting Classification
0.9268928936601293
Gradient Boosting Classification2
0.9151000081879276
Gradient Boosting Classification3
0.9457520001133995
Random Forest Classification
0.954496512164248
Random Forest Classification2
0.954496512164248
Gradient Boosting Classification
0.9497066713394321
Gradient Boosting Classification2
0.9568730125014238
Gradient Boosting Classification3
0.9498223699715677
Random Forest Classification


/var/folders/k9/6c26p32970l0vhs0_8nyb60h0000gn/T/ipykernel_31508/130579575.py:33: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_final_pred = filtered_nat_df.groupby(['age_recoded','race_recoded', 'male','education_recoded'], )['predicted_vote'].value_counts().unstack(fill_value=0)


0.9136787173922453
Random Forest Classification2
0.9091149198116959
Gradient Boosting Classification
0.916083916083916
Gradient Boosting Classification2
0.8950146562878659
Gradient Boosting Classification3
0.9066508371708336


/var/folders/k9/6c26p32970l0vhs0_8nyb60h0000gn/T/ipykernel_31508/130579575.py:33: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_final_pred = filtered_nat_df.groupby(['age_recoded','race_recoded', 'male','education_recoded'], )['predicted_vote'].value_counts().unstack(fill_value=0)


In [151]:
all_genders = [True, False ]
all_ages = [1,2, 3]
all_educations = [1,3]
all_races = [1,2, 3,4,9]
all_combinations = pd.MultiIndex.from_product([all_ages,all_races, all_genders, all_educations], names=['age_recoded','race_recoded', 'male','education_recoded']).to_frame(index=False)
mar_pred = pd.merge(all_combinations,mar_pred, on=['age_recoded','race_recoded', 'male','education_recoded'], how='outer')
june_pred= pd.merge(all_combinations,june_pred, on=['age_recoded','race_recoded', 'male','education_recoded'], how='outer')
aug_pred = pd.merge(all_combinations,aug_pred, on=['age_recoded','race_recoded', 'male','education_recoded'], how='outer')
merge_mar_june = pd.merge(mar_pred, june_pred, on=['age_recoded',	'race_recoded',	'male', 'education_recoded'], how='outer').fillna(0)
merge_mar_june['0'] = merge_mar_june['0_x'] + merge_mar_june['0_y']
# add 1_x and 1_y together
merge_mar_june['1'] = merge_mar_june['1_x'] + merge_mar_june['1_y']
# drop the _x and _y columns
merge_mar_june = merge_mar_june.drop(columns = ['0_x', '0_y', '1_x', '1_y'])

# add the propensity vibe column
merge_mar_june['propensity_t_pred'] = merge_mar_june['1'] / (merge_mar_june['0'] + merge_mar_june['1'])
merge_mar_june

/var/folders/k9/6c26p32970l0vhs0_8nyb60h0000gn/T/ipykernel_31508/2350480096.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merge_mar_june = pd.merge(mar_pred, june_pred, on=['age_recoded',	'race_recoded',	'male', 'education_recoded'], how='outer').fillna(0)


,age_recoded,race_recoded,male,education_recoded,0,1,propensity_t_pred
0,1,1,False,1,3.0,3.0,0.500000
1,1,1,False,3,26.0,16.0,0.380952
2,1,1,True,1,7.0,11.0,0.611111
3,1,1,True,3,28.0,22.0,0.440000
4,1,2,False,1,1.0,0.0,0.000000
5,1,2,False,3,4.0,0.0,0.000000
6,1,2,True,1,1.0,1.0,0.500000
7,1,2,True,3,5.0,1.0,0.166667
8,1,3,False,1,0.0,0.0,NaN
9,1,3,False,3,5.0,0.0,0.000000


In [152]:

merged_df = merge_mar_june.copy()
post_strat = pd.read_csv("../data/post_stratification_data_by_state.csv")
propensity = pd.read_csv("../data/output11.csv")
megaStrat = pd.merge(propensity, post_strat, how='left', on=['STATEFIP',	'age_recoded',	'race_recoded'	,'male'	,'education_recoded'])
megaStrat = megaStrat.dropna()

In [153]:
# propensity = pd.read_csv("../data/output7.csv") 
propensity = pd.read_csv("../data/output11.csv")
# 7 is best so far
# propensity = pd.read_csv("../data/output10.csv")

# post_strat = post_strat.drop(columns = ['prop'])
# # propensity = propensity.drop(columns = ['mrp_subgroup_estimate_se'])
# propensity_strat = pd.merge(propensity, post_strat)
# multiply by 0.595 if male, 0.63 if female
propensity_strat = megaStrat.copy()

propensity_strat['PERWT_scaled'] = propensity_strat.apply(
    lambda row: row['PERWT'] * 0.595 if row['male'] else row['PERWT'] * 0.63, axis=1
)



multipliers_age = {
    1: 0.55,
    2: 0.656,
    3: 0.73,
}
# White 69.8, black 58.7, asian 42.8, latino 38.8
# 2018 57.5, 51.4, 40.4, 40.3

multipliers = {
    1: 0.575,
    2: 0.514,
    3: 0.404,
    4: 0.403,
    9: 0.5
}

# make perwt scaled multiplied if its white black asian or latino
propensity_strat['PERWT_scaled_race'] = propensity_strat.apply(lambda row: row['PERWT'] * multipliers[row['race_recoded']], axis=1)

propensity_strat['PERWT_scaled_age'] = propensity_strat.apply(lambda row: row['PERWT'] * multipliers_age[row['age_recoded']], axis=1)

# propensity_strat['PERWT_scaled'] = propensity_strat.apply(
# propensity_strat['PERWT_scaled'] = propensity_strat.apply(
#     lambda row: row['PERWT'] * 0.698 if row['race_recoded'] == 1 elif row['PERWT'] * 0.698 if row['race_recoded'] == 1 else row['PERWT'] * 0.63, axis=1
# )
propensity_strat['trumpVote'] = propensity_strat['mrp_subgroup_estimate'] * propensity_strat['PERWT_scaled']
propensity_strat['bidenVote'] = (1 - propensity_strat['mrp_subgroup_estimate']) * propensity_strat['PERWT_scaled']
propensity_strat
print('trump percent', propensity_strat['trumpVote'].sum() / (propensity_strat['trumpVote'].sum() + propensity_strat['bidenVote'].sum()), 'trumpSum', propensity_strat['trumpVote'].sum())
print('biden percent', propensity_strat['bidenVote'].sum() / (propensity_strat['trumpVote'].sum() + propensity_strat['bidenVote'].sum()), 'bidenSum', propensity_strat['bidenVote'].sum())
propensity_strat

trump percent 0.4910874839836854 trumpSum 75401820.23110397
biden percent 0.5089125160163146 bidenSum 78138684.64889602


,age_recoded,race_recoded,male,education_recoded,STATEFIP,mrp_subgroup_estimate,mrp_subgroup_estimate_se,PERWT,prop,PERWT_scaled,PERWT_scaled_race,PERWT_scaled_age,trumpVote,bidenVote
0,1,1,False,1,1,0.679457,0.080854,327170.0,1.291648e-03,206117.100,188122.750,179943.50,140047.661156,66069.438844
1,1,1,False,1,2,0.732036,0.082192,47024.0,1.856480e-04,29625.120,27038.800,25863.20,21686.663943,7938.456057
2,1,1,False,1,4,0.579128,0.077876,358383.0,1.414875e-03,225781.290,206070.225,197110.65,130756.371494,95024.918506
3,1,1,False,1,5,0.620041,0.121134,221846.0,8.758351e-04,139762.980,127561.450,122015.30,86658.734910,53104.245090
4,1,1,False,1,6,0.388438,0.072786,1424354.0,5.623267e-03,897343.020,819003.550,783394.70,348562.253953,548780.766047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2934,3,9,True,3,50,0.433989,0.118686,11.0,4.342736e-08,6.545,5.500,8.03,2.840457,3.704543
2935,3,9,True,3,51,0.600421,0.116523,556.0,2.195056e-06,330.820,278.000,405.88,198.631124,132.188876
2936,3,9,True,3,53,0.498659,0.119723,342.0,1.350196e-06,203.490,171.000,249.66,101.472189,102.017811
2937,3,9,True,3,54,0.631852,0.113445,77.0,3.039915e-07,45.815,38.500,56.21,28.948309,16.866691


In [176]:
propensity_strat = pd.merge(propensity_strat, merged_df, on = ['age_recoded', 'education_recoded','male', 'race_recoded'], how='left')
propensity_strat['ML_pred'] = propensity_strat['propensity_t_pred']

propensity_strat['ML_pred'] = propensity_strat['ML_pred'].fillna(propensity_strat['mrp_subgroup_estimate'])


KeyError: 'propensity_t_pred'

In [179]:
# propensity_strat['ML_pred'] = merged_df['propensity_t_pred'].values
# fill with the originall propensity score if 0
propensity_strat['ML_pred'] = propensity_strat['ML_pred'].fillna(propensity_strat['mrp_subgroup_estimate'])
# if ML pred is 0, fill with MRP pred / 2

propensity_strat.loc[propensity_strat['ML_pred'] == 0, 'ML_pred'] = propensity_strat['mrp_subgroup_estimate'] / 8
propensity_strat.loc[propensity_strat['ML_pred'] == 1, 'ML_pred'] = 1 - (propensity_strat['mrp_subgroup_estimate'] / 6 )

#weight the mrp and ml predictions by the PERWT_scaled
propensity_strat['mrp_and_ML'] = ((propensity_strat['mrp_subgroup_estimate'] * 5) + propensity_strat['ML_pred']) / 6
propensity_strat
propensity_strat['trumpVote_altered'] = propensity_strat['mrp_and_ML'] * propensity_strat['PERWT_scaled']
propensity_strat['bidenVote_altered'] = (1 - propensity_strat['mrp_and_ML']) * propensity_strat['PERWT_scaled']
print('trump sum', propensity_strat['trumpVote_altered'].sum(), 'biden sum', propensity_strat['bidenVote_altered'].sum())
print('trump percent', propensity_strat['trumpVote_altered'].sum() / (propensity_strat['trumpVote_altered'].sum() + propensity_strat['bidenVote_altered'].sum()))
print('biden percent', propensity_strat['bidenVote_altered'].sum() / (propensity_strat['trumpVote_altered'].sum() + propensity_strat['bidenVote_altered'].sum()))


propensity_strat_breakdown = propensity_strat[['male', 'age_recoded','race_recoded', 'education_recoded', 'mrp_subgroup_estimate', 'ML_pred', 'mrp_and_ML', 'STATEFIP']]


trump sum 76702452.95854497 biden sum 76838051.92145503
trump percent 0.49955842608757856
biden percent 0.5004415739124215


In [181]:
census_by_state = pd.read_csv("../data/post_stratification_data_by_state.csv")
census_by_state
elec_votes2 = pd.read_csv("../data/2020_electoral_results.csv")
elec_votes = pd.read_csv("../data/2020_ecollege_rep.csv")
elec_votes
# merge the two on stateFIP
census_by_state = pd.merge(census_by_state, elec_votes, left_on = 'STATEFIP', right_on = 'STATEFP')
census_by_state = pd.merge(census_by_state, elec_votes2, left_on = 'state', right_on = 'State')
# White 69.8, black 58.7, asian 42.8, latino 38.8

# multipliers = {
#     1: 0.698,
#     2: 0.587,
#     3: 0.428,
#     4: 0.388,
#     9: 1
# }

# # make perwt scaled multiplied if its white black asian or latino
# census_by_state['PERWT_scaled'] = census_by_state.apply(lambda row: row['PERWT'] * multipliers[row['race_recoded']], axis=1)
census_by_state['PERWT_scaled'] = census_by_state.apply(
    lambda row: row['PERWT'] * 0.595 if row['male'] else row['PERWT'] * 0.63, axis=1
)
# map the propensity strat to the census data by education, age, race, gender
census_by_state = pd.merge(census_by_state, propensity_strat_breakdown, on = ['age_recoded', 'education_recoded','male', 'race_recoded', 'STATEFIP' ])
# # census_by_state['mrp_and_ML'] = propensity_strat['mrp_and_ML']
# census_by_state
census_by_state

,STATEFIP,age_recoded,race_recoded,male,education_recoded,PERWT,prop,STATE,STATEFP,STATENS,...,State,state_votes,biden,trump,harris,pence,PERWT_scaled,mrp_subgroup_estimate,ML_pred,mrp_and_ML
0,1,1,1,False,1,327170.0,1.291648e-03,AL,1,1779775,...,alabama,9,0,9,0,9,206117.100,0.679457,0.500000,0.649547
1,1,1,1,False,3,1916.0,7.564257e-06,AL,1,1779775,...,alabama,9,0,9,0,9,1207.080,0.566171,0.380952,0.535301
2,1,1,1,True,1,330345.0,1.304183e-03,AL,1,1779775,...,alabama,9,0,9,0,9,196555.275,0.748718,0.611111,0.725783
3,1,1,1,True,3,2014.0,7.951155e-06,AL,1,1779775,...,alabama,9,0,9,0,9,1198.330,0.646978,0.440000,0.612482
4,1,1,2,False,1,171855.0,6.784736e-04,AL,1,1779775,...,alabama,9,0,9,0,9,108268.650,0.219972,0.027496,0.187892
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2873,56,3,4,True,1,4098.0,1.617867e-05,WY,56,1779807,...,wyoming,3,0,3,0,3,2438.310,0.777065,0.666667,0.758665
2874,56,3,4,True,3,197.0,7.777446e-07,WY,56,1779807,...,wyoming,3,0,3,0,3,117.215,0.684754,0.500000,0.653962
2875,56,3,9,False,1,2080.0,8.211719e-06,WY,56,1779807,...,wyoming,3,0,3,0,3,1310.400,0.811494,0.500000,0.759578
2876,56,3,9,False,3,48.0,1.895012e-07,WY,56,1779807,...,wyoming,3,0,3,0,3,30.240,0.729696,0.250000,0.649746


In [182]:
turnout_state = pd.read_csv("../data/turnout_by_state.csv")
turnout_state['STATE_NAME'] = turnout_state['State'].str.lower()
census_by_state2 = pd.merge(census_by_state, turnout_state, on = ['STATE_NAME'])
census_by_state2['PERWT_state_scaled'] = census_by_state2['PERWT_scaled'] / 100 * census_by_state2['2020_turnout']

In [183]:
allStates = pd.read_csv("../data/output11.csv")

elec_votes = pd.read_csv("../data/2020_electoral_results.csv")
elec_votes
# merge the two on stateFIP
allStates = pd.merge(allStates, census_by_state,  on = ['age_recoded', 'education_recoded','male', 'race_recoded', 'STATEFIP'])
# allStates = pd.merge(allStates, elec_votes, left_on = 'STATEFIP', right_on = 'STATEFP')
# census_by_state = pd.merge(census_by_state, elec_votes, left_on = 'STATEFIP', right_on = 'STATEFP')
turnout_state = pd.read_csv("../data/turnout_by_state.csv")
turnout_state['STATE_NAME'] = turnout_state['State'].str.lower()
# allStates['mrp_subgroup_estimate_x'].fillna(allStates['mrp_subgroup_estimate_y'], inplace = True)
# allStates['PERWT_scaled'] = allStates.apply(
#     lambda row: row['PERWT'] * 0.595 if row['male'] else row['PERWT'] * 0.63, axis=1
# )
allStates = pd.merge(allStates, turnout_state, on = ['STATE_NAME'])
allStates['PERWT_state_scaled'] = allStates['PERWT'] / 100 * allStates['2020_turnout']
allStates['PERWT_scaled'] = allStates.apply(
    lambda row: row['PERWT'] * 0.595 if row['male'] else row['PERWT'] * 0.63, axis=1
)
allStates

,age_recoded,race_recoded,male,education_recoded,STATEFIP,mrp_subgroup_estimate_x,mrp_subgroup_estimate_se,PERWT,prop,STATE,...,trump,harris,pence,PERWT_scaled,mrp_subgroup_estimate_y,ML_pred,mrp_and_ML,State_y,2020_turnout,PERWT_state_scaled
0,1,1,False,1,1,0.679457,0.080854,327170.0,1.291648e-03,AL,...,9,0,9,206117.100,0.679457,0.500000,0.649547,Alabama,60.5,197937.850
1,1,1,False,1,2,0.732036,0.082192,47024.0,1.856480e-04,AK,...,3,0,3,29625.120,0.732036,0.500000,0.693364,Alaska,63.8,30001.312
2,1,1,False,1,4,0.579128,0.077876,358383.0,1.414875e-03,AZ,...,0,11,0,225781.290,0.579128,0.500000,0.565940,Arizona,71.9,257677.377
3,1,1,False,1,5,0.620041,0.121134,221846.0,8.758351e-04,AR,...,6,0,6,139762.980,0.620041,0.500000,0.600034,Arkansas,54.0,119796.840
4,1,1,False,1,6,0.388438,0.072786,1424354.0,5.623267e-03,CA,...,0,55,0,897343.020,0.388438,0.500000,0.407032,California,65.1,927254.454
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2873,3,9,True,3,50,0.433989,0.118686,11.0,4.342736e-08,VT,...,0,3,0,6.545,0.433989,0.666667,0.472769,Vermont,68.4,7.524
2874,3,9,True,3,51,0.600421,0.116523,556.0,2.195056e-06,VA,...,0,13,0,330.820,0.600421,0.666667,0.611462,Virginia,71.5,397.540
2875,3,9,True,3,53,0.498659,0.119723,342.0,1.350196e-06,WA,...,0,12,0,203.490,0.498659,0.666667,0.526661,Washington,71.5,244.530
2876,3,9,True,3,54,0.631852,0.113445,77.0,3.039915e-07,WV,...,5,0,5,45.815,0.631852,0.666667,0.637655,West Virginia,56.1,43.197


# Predict With ML

In [184]:
# group by states and then predict if the state will go red or blue
census_by_state = allStates.copy()
# census_by_state['bothScores'] = census_by_state['mrp_subgroup_estimate_x'] 
census_by_state['bothScores'] = ((census_by_state['mrp_subgroup_estimate_x'] * 6 ) + census_by_state['ML_pred']) / 7

census_by_state['trump_votes_states'] = census_by_state['bothScores'] * census_by_state['PERWT_scaled']
census_by_state['biden_votes_states'] = (1 - census_by_state['bothScores']) * census_by_state['PERWT_scaled']
# sum up total trump/biden votes 

census_by_state['state_demo_ratio'] =  census_by_state['trump_votes_states'] / (census_by_state['trump_votes_states'] + census_by_state['biden_votes_states'] )
# census_by_state['mrp_and_ML'].value_counts()
elec_college_pred = census_by_state.groupby('state')[['trump_votes_states','biden_votes_states'] ].sum()
# if trump votes > biden votes, then trump wins the state
elec_college_pred['state_pred'] = elec_college_pred.apply(lambda row: 1 if row['trump_votes_states'] > row['biden_votes_states'] else 0, axis = 1)
elec_college_pred
final_pred = pd.merge(elec_college_pred, elec_votes[:51], left_on = 'state', right_on = 'State')

print('trump sum', final_pred['trump_votes_states'].sum(), 'biden sum', final_pred['biden_votes_states'].sum())
final_pred['state_votes'] = final_pred['state_votes'].astype(int)
# sum up electoral votes for all states with pred of 1 and 0
final_pred.groupby('state_pred')['state_votes'].sum()

trump sum 76516648.28319626 biden sum 77023856.59680372


state_pred
0    262
1    267
Name: state_votes, dtype: int64

In [185]:
final_pred

,trump_votes_states,biden_votes_states,state_pred,State,state_votes,biden,trump,harris,pence
0,1.362572e+06,9.684033e+05,1,alabama,9,0,9,0,9
1,2.455450e+05,9.295124e+04,1,alaska,3,0,3,0,3
2,1.976441e+06,1.414729e+06,1,arizona,11,11,0,11,0
3,8.334508e+05,5.821084e+05,1,arkansas,6,0,6,0,6
4,7.183241e+06,1.144028e+07,0,california,55,55,0,55,0
5,1.495187e+06,1.215983e+06,1,colorado,9,9,0,9,0
6,7.298559e+05,1.008593e+06,0,connecticut,7,7,0,7,0
7,2.144064e+05,2.536703e+05,0,delaware,3,3,0,3,0
8,6.303317e+04,2.904634e+05,0,district of columbia,3,3,0,3,0
9,5.488100e+06,4.935879e+06,1,florida,29,0,29,0,29


# no ML

In [175]:
# group by states and then predict if the state will go red or blue
census_by_state = allStates.copy()
census_by_state['bothScores'] = census_by_state['mrp_subgroup_estimate_x'] 
# census_by_state['bothScores'] = ((census_by_state['mrp_subgroup_estimate_x'] *2 ) + census_by_state['ML_pred']) / 3

census_by_state['trump_votes_states'] = census_by_state['bothScores'] * census_by_state['PERWT_scaled']
census_by_state['biden_votes_states'] = (1 - census_by_state['bothScores']) * census_by_state['PERWT_scaled']
# sum up total trump/biden votes 

census_by_state['state_demo_ratio'] =  census_by_state['trump_votes_states'] / (census_by_state['trump_votes_states'] + census_by_state['biden_votes_states'] )
# census_by_state['mrp_and_ML'].value_counts()
elec_college_pred = census_by_state.groupby('state')[['trump_votes_states','biden_votes_states'] ].sum()
# if trump votes > biden votes, then trump wins the state
elec_college_pred['state_pred'] = elec_college_pred.apply(lambda row: 1 if row['trump_votes_states'] > row['biden_votes_states'] else 0, axis = 1)
elec_college_pred
final_pred = pd.merge(elec_college_pred, elec_votes2, left_on = 'state', right_on = 'State')
# final_pred = elec_college_pred
print('trump sum', final_pred['trump_votes_states'].sum(), 'biden sum', final_pred['biden_votes_states'].sum())
final_pred['state_votes'] = final_pred['state_votes'].astype(int)
# sum up electoral votes for all states with pred of 1 and 0
final_pred.groupby('state_pred')['state_votes'].sum()

trump sum 75401820.23110399 biden sum 78138684.64889602


state_pred
0    262
1    267
Name: state_votes, dtype: int64

In [162]:
final_pred

,trump_votes_states,biden_votes_states,state_pred,State,state_votes,biden,trump,harris,pence
0,1.390886e+06,9.400893e+05,1,alabama,9,0,9,0,9
1,2.528480e+05,8.564825e+04,1,alaska,3,0,3,0,3
2,1.981145e+06,1.410025e+06,1,arizona,11,11,0,11,0
3,8.397014e+05,5.758578e+05,1,arkansas,6,0,6,0,6
4,6.836219e+06,1.178731e+07,0,california,55,55,0,55,0
5,1.476037e+06,1.235134e+06,1,colorado,9,9,0,9,0
6,6.895890e+05,1.048860e+06,0,connecticut,7,7,0,7,0
7,2.099359e+05,2.581408e+05,0,delaware,3,3,0,3,0
8,5.119447e+04,3.023021e+05,0,district of columbia,3,3,0,3,0
9,5.471424e+06,4.952555e+06,1,florida,29,0,29,0,29
